In [4]:
import numpy as np
import pandas as pd
import feather
from pandasql import sqldf
from fisher import pvalue
from scipy import stats
from pprint import pprint

In [5]:
def simu_toy(n_pos, n_max, causal, n=50, const=1, seed=999):
    ptn_ls = []
    if n_pos <= n_max:
        ptn_keys = ["p{}".format(i+1) for i in range(n_pos*2)]
        for i in range(n_pos):
            ptn = [1]*(i+1) + [0]*(n_pos-i-1)
            ptn_ls.append(ptn)
        for i in range(n_pos, n_pos*2):
            ptn = [0]*(i-n_pos+1) + [1]*(2*n_pos-i-1)
            ptn_ls.append(ptn)
    else:
        ptn_keys = ["p{}".format(i+1) for i in range(n_pos+n_max)]
        for i in range(n_max):
            ptn = [1]*(i+1) + [0]*(n_pos-i-1)
            ptn_ls.append(ptn)
        for i in range(n_max, n_pos):
            ptn = [0]*(i-n_max+1) + [1]*n_max + [0]*(n_pos-i-1)
            ptn_ls.append(ptn)
        for i in range(n_pos, n_pos+n_max):
            ptn = [0]*(i-n_max+1) + [1]*(n_pos+n_max-i-1)
            ptn_ls.append(ptn)

    ptn_dict = dict(zip(ptn_keys, ptn_ls))
    n_ls = [n]*len(ptn_keys)
    n_dict = dict(zip(ptn_keys, n_ls))
    sample = sum([[ptn_dict[i]] * n_dict[i] for i in n_dict.keys()], [])
    
    np.random.seed(seed)    
    mat = [[np.random.normal(0,1)] + line for line in sample]
    config = []
    for line in mat:
        if line[causal] == 1:
            line[0] = line[0] + const
            config.append(line[0])
        else:
            config.append(line[0])
        if line[0] > np.median(config):
            line[0] = 1
        else:
            line[0] = 0
    mat = np.matrix(mat)
    df = pd.DataFrame(mat, columns = ["phenotype"] + ["pos{}".format(i+1) for i in range(mat.shape[1]-1)])
    counts = df.groupby(["pos{}".format(i+1) for i in range(mat.shape[1]-1)] + ["phenotype"]).size()
    for col in df.columns:
        df[col] = df[col].astype(np.uint8)
    output = feather.write_dataframe(df, "data/toy_pos{}_causal{}_n{}.feather".format(n_pos, causal, sum(n_ls)))
    return df, counts

In [6]:
df, counts = simu_toy(n_pos=15, n_max=10, causal=8)
print (df)

      phenotype  pos1  pos2  pos3  pos4  pos5  pos6  pos7  pos8  pos9  pos10  \
0             0     1     1     1     1     1     1     1     0     0      0   
1             1     1     1     1     1     1     1     1     0     0      0   
2             0     1     1     1     1     1     1     1     0     0      0   
3             0     1     1     1     1     1     1     1     0     0      0   
4             0     1     1     1     1     1     1     1     0     0      0   
5             0     1     1     1     1     1     1     1     0     0      0   
6             1     1     1     1     1     1     1     1     0     0      0   
7             0     1     1     1     1     1     1     1     0     0      0   
8             1     1     1     1     1     1     1     1     0     0      0   
9             1     1     1     1     1     1     1     1     0     0      0   
10            0     1     1     1     1     1     1     1     0     0      0   
11            1     1     1     1     1 

# mat_l = sum([[ptn_dict[i]] * n_dict[i] for i in n_dict.keys()], [])
print(len(mat_l))

In [ ]:
mat_1=[]
for i in n_dict.keys():
    for j in range(n_dict[i]):
        mat_1.append(ptn_dict[i])
print ((mat_1))

In [ ]:
n_gene = 15
n_max = 10
ptn_dict, n_dict = simu_toy(n_pos=n_gene, n_max=n_max)
# print ([n_dict.keys()])

[ptn_dict[i] for j in range(n_dict[i]) for i in n_dict.keys()]

In [ ]:
seed = 999
np.random.seed(seed)
config = [[np.random.normal(0,1)]+ptn_dict[i] for j in range(n_dict[i]) for i in n_dict.keys()]
print (config)
# first gene is causal
for line in config:
    if line[1]==1:
        line[0] = line[0]+1
# print (config)

In [ ]:
def toy_10gene(ptn_dict, n_dict, const=1, seed=999):
    np.random.seed(seed)
    return None

In [14]:
sum([1,3,54,6,7,8])

79